In [1]:
#Utilizzeremo in modo effettivo i modelli definiti prima per il riconoscimento tra cani e gatti

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')



3-conv-64-nodes-0-dense-1552207388
Instructions for updating:
Colocations handled automatically by placer.
Train on 17439 samples, validate on 7475 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
17439/17439 [==============================] - 313s 18ms/sample - loss: 0.6529 - acc: 0.6098 - val_loss: 0.5851 - val_acc: 0.6981
Epoch 2/10
17439/17439 [==============================] - 279s 16ms/sample - loss: 0.5668 - acc: 0.7043 - val_loss: 0.5394 - val_acc: 0.7344
Epoch 3/10
17439/17439 [==============================] - 326s 19ms/sample - loss: 0.5144 - acc: 0.7485 - val_loss: 0.5038 - val_acc: 0.7624
Epoch 4/10
17439/17439 [==============================] - 364s 21ms/sample - loss: 0.4746 - acc: 0.7750 - val_loss: 0.4848 - val_acc: 0.7651
Epoch 5/10
17439/17439 [==============================] - 430s 25ms/sample - loss: 0.4507 - acc: 0.7880 - val_loss: 0.4564 - val_acc: 0.7826
Epoch 6/10
17439/17439 [==============================] - 377s 22ms/sample - loss: 0.4202 -

In [3]:
#PER VALUTARE L'EFFETTIVA EFFICENZA DEL CLASSIFICATORE VENGONO UTILIZZATE NELLA FASE DI TEST DUE IMMAGINI 
#CHE NON FANNO PARTE DEL TRAINING SET

In [7]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]  # will use this to convert prediction num to string value


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.



In [8]:
model = tf.keras.models.load_model("64x3-CNN.model")

In [9]:


prediction = model.predict([prepare('Prova1.jpg')])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT



In [10]:
prediction

array([[1.]], dtype=float32)

In [11]:
prediction[0][0]

1.0

In [12]:
print(CATEGORIES[int(prediction[0][0])])

Cat


In [13]:
prediction = model.predict([prepare('Prova2.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[0.]]
Dog


In [15]:
import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]


def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.model")

prediction = model.predict([prepare('Prova2.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[0.]]
Dog
